<a href="https://colab.research.google.com/github/enzoampil/data-science-demos/blob/master/ml-deployment/how_to_deply_ml_models_lorenzo_ampil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [31]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install streamlit
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

--2020-09-25 10:55:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.170.115.131, 52.206.15.164, 54.84.116.182, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.170.115.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  53.7MB/s    in 0.2s    

2020-09-25 10:55:34 (53.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]



In [ ]:
import spacy

# How to deploy ML models walkthrough


1. **Realtime deployment** - you run the model when a specific event happens
2. **Batch deployment** - you run the model periodically


## Let's use a simple entity detector using spacy

In [4]:
def extract_entities(text):
# Load English tokenizer, tagger, parser, NER and word vectors
    nlp = spacy.load("en_core_web_sm")


    doc = nlp(text)

    # Find named entities, phrases and concepts
    entities = []
    for entity in doc.ents:
        entities.append((entity.text, entity.label_))
    return entities

In [5]:
# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")

extract_entities(text)

[('Sebastian', 'NORP'),
 ('Google', 'ORG'),
 ('2007', 'DATE'),
 ('American', 'NORP'),
 ('Recode', 'ORG'),
 ('earlier this week', 'DATE')]

# Realtime deployment

1. Typical process
    1. You want predictions after a certain "event" happens
    2. I.e., the predictions are "event driven"
    3. Examples of "events
        1. Someone clicks on the signup button
            1. What product plan to recommend them???
        2. You book a trip on grab
            1. What's the ETA of the trip???
2. How to implement
    1. Deploy as an API
        1. I recommend doing this with FastAPI
    2. Deploy straight into a webapp
        1. Streamlit is the easiest way

### Deploy on FastAPI

In [33]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/{text}')
async def extract_entities(text):
# Load English tokenizer, tagger, parser, NER and word vectors
    nlp = spacy.load("en_core_web_sm")


    doc = nlp(text)

    # Find named entities, phrases and concepts
    entities = []
    for entity in doc.ents:
        entities.append((entity.text, entity.label_))
    return entities

In [43]:
# Have to translate spaces to %20 to use as URL argument
translated_input = text.replace(" ", "%20")
print(translated_input)

When%20Sebastian%20Thrun%20started%20working%20on%20self-driving%20cars%20at%20Google%20in%202007,%20few%20people%20outside%20of%20the%20company%20took%20him%20seriously.%20“I%20can%20tell%20you%20very%20senior%20CEOs%20of%20major%20American%20car%20companies%20would%20shake%20my%20hand%20and%20turn%20away%20because%20I%20wasn’t%20worth%20talking%20to,”%20said%20Thrun,%20in%20an%20interview%20with%20Recode%20earlier%20this%20week.


In [65]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

url = ngrok.connect(port=8000)
print('Public URL:', url)
print('Sample URL w/ input:', url + "/" + translated_input)
nest_asyncio.apply()

uvicorn.run(app, port=8000)

Public URL: http://48e55a31f05d.ngrok.io
Sample URL w/ input: http://48e55a31f05d.ngrok.io/When%20Sebastian%20Thrun%20started%20working%20on%20self-driving%20cars%20at%20Google%20in%202007,%20few%20people%20outside%20of%20the%20company%20took%20him%20seriously.%20“I%20can%20tell%20you%20very%20senior%20CEOs%20of%20major%20American%20car%20companies%20would%20shake%20my%20hand%20and%20turn%20away%20because%20I%20wasn’t%20worth%20talking%20to,”%20said%20Thrun,%20in%20an%20interview%20with%20Recode%20earlier%20this%20week.


INFO:     Started server process [100]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     112.207.114.125:0 - "GET /When%20Sebastian%20Thrun%20started%20working%20on%20self-driving%20cars%20at%20Google%20in%202007%2C%20few%20people%20outside%20of%20the%20company%20took%20him%20seriously.%20 HTTP/1.1" 200 OK
INFO:     112.207.114.125:0 - "GET /favicon.ico HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [100]


### Deploy on Streamlit

In [46]:
STREAMLIT_CODE = """
import streamlit as st
import spacy

def extract_entities(text):
    # Load English tokenizer, tagger, parser, NER and word vectors
    nlp = spacy.load("en_core_web_sm")


    doc = nlp(text)

    # Find named entities, phrases and concepts
    entities = []
    for entity in doc.ents:
        entities.append((entity.text, entity.label_))
    return entities

user_input = st.text_input("Input text", "")

st.text(extract_entities(user_input))
"""

In [47]:
#!echo {STREAMLIT_CODE} >> spacy_streamlit.py
st_file = open("spacy_streamlit.py", "w")
n = st_file.write(STREAMLIT_CODE)
st_file.close()

In [48]:
!cat spacy_streamlit.py


import streamlit as st
import spacy

def extract_entities(text):
    # Load English tokenizer, tagger, parser, NER and word vectors
    nlp = spacy.load("en_core_web_sm")


    doc = nlp(text)

    # Find named entities, phrases and concepts
    entities = []
    for entity in doc.ents:
        entities.append((entity.text, entity.label_))
    return entities

user_input = st.text_input("Input text", "")

st.text(extract_entities(user_input))


**Use the output of this command as the link to your app.**

In [67]:
# If it says "list index out of range" just run it again
get_ipython().system_raw('./ngrok http 8501 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://3d0bd5c624a7.ngrok.io


In [68]:
!streamlit run spacy_streamlit.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.73.210.146:8501

  Stopping...
  Stopping...


## Batch deployment
1. Typical process
    1. Periodic ingestion from a data source
        1. This is when you want predictions periodically
            1. Return the list of users most likely to "churn" every week
        2. Example data sources
            1. App database (DB)
            2. External APIs (e.g. Twitter)
            3. Some website you're scraping
    2. Use data as an input into an ML pipeline
        1. Extract entities from using NER
        2. Predict demographics of new users based on their signup details? Creepy ...
        3. Classify article topics based on their contents
2. How to implement
    1. Workflow orchestration frameworks
        1. Crontab
            1. Use this if you want something super fast
            2. All you need to do is edit a file and use cron notation
                1. (minute / hour / day of month / month / day of week)
                2. `* * * * *` - every minute of everyday
                3. `5 4 * * *` - 4:05 AM every day
        2. Airflow
            1. Use this when your pipeline becomes more complex
                1. Multiple dependencies
            2. It's a pain to set up but well worth it once you have it up and running
        3. Dagster*
            1. New framework that I've heard great things about, but haven't tried!!

In [23]:
!pip install \
 apache-airflow==1.10.12 \
 --constraint "https://raw.githubusercontent.com/apache/airflow/constraints-1.10.12/constraints-3.7.txt"

     |████████████████████████████████| 4.7MB 2.9MB/s 
     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 1.1MB 38.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 829kB 43.0MB/s 
     |████████████████████████████████| 327kB 43.0MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 1.7MB 36.1MB/s 
     |████████████████████████████████| 133kB 47.7MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 1.2MB 39.3MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 1.7MB 39.9MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 460kB 24.2MB/s 
     |████████████████████████████████| 10.5MB 

In [52]:
!airflow initdb

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
DB: sqlite:////root/airflow/airflow.db
[2020-09-25 11:08:37,735] {db.py:378} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> e3a246e0dc1, current schema
INFO  [alembic.runtime.migration] Running upgrade e3a246e0dc1 -> 1507a7289a2f, create is_encrypted
/usr/local/lib/python3.6/dist-packages/alembic/ddl/sqlite.py:41: UserWarning: Skipping unsupported ALTER for creation of implicit constraintPlease refer to the batch mode feature which allows for SQLite migrations using a copy-and-move strategy.
  "Skipping unsupport

**Use the output of this command as the link to your app.**

In [73]:
# If it says "list index out of range" just run it again
get_ipython().system_raw('./ngrok http 8501 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [72]:
!airflow webserver -p 8501

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
[2020-09-25 11:35:42,943] {__init__.py:50} INFO - Using executor SequentialExecutor
[2020-09-25 11:35:42,944] {dagbag.py:417} INFO - Filling up the DagBag from /root/airflow/dags
Running the Gunicorn Server with:
Workers: 4 sync
Host: 0.0.0.0:8501
Timeout: 120
Logfiles: - -
[2020-09-25 11:35:44 +0000] [1644] [INFO] Starting gunicorn 20.0.4
[2020-09-25 11:35:44 +0000] [1644] [INFO] Listening at: http://0.0.0.0:8501 (1644)
[2020-09-25 11:

## Sorry! We don't have enough time to make our own DAG here (timeboxed myself to an hour), but here's a good tutorial on the [topic](http://michal.karzynski.pl/blog/2017/03/19/developing-workflows-with-apache-airflow/) :)